In [ ]:
!pip install tensorflow tf-agents --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 23.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.3 MB/s eta 0:00:00


In [ ]:
!pip install atari_py --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.6/540.6 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import tensorflow as tf
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.drivers import dynamic_step_driver
from tf_agents.trajectories import trajectory

# Define the environment
env_name = 'CartPole-v1'
train_env = tf_py_environment.TFPyEnvironment(suite_gym.load(env_name))
eval_env = tf_py_environment.TFPyEnvironment(suite_gym.load(env_name))

# Define the Q-network
fc_layer_params = (200, 100)  # Add more layers or neurons


q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params
)

# Define the DQN agent
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-4)
train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter
)

agent.initialize()

# Define the replay buffer and collect policy
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=50000
)

collect_policy = agent.collect_policy
time_step = train_env.reset()
collect_policy_state = collect_policy.get_initial_state(train_env.batch_size)

for _ in range(1):  # Adjust the num_steps as needed
    action_step = collect_policy.action(time_step, collect_policy_state)
    next_time_step = train_env.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)
    replay_buffer.add_batch(traj)

    time_step = next_time_step
    collect_policy_state = action_step.state


# Define the dataset
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=64,
    num_steps=2
).prefetch(3)

iterator = iter(dataset)
num_iterations = 10000
# ...

# Continue the training process
for iteration in range(num_iterations):  # Use a different variable name
    # Manually collect experience and add it to the replay buffer
    time_step = train_env.reset()
    collect_policy_state = collect_policy.get_initial_state(train_env.batch_size)

    for _ in range(1):  # Adjust the num_steps as needed
        action_step = collect_policy.action(time_step, collect_policy_state)
        next_time_step = train_env.step(action_step.action)
        traj = trajectory.from_transition(time_step, action_step, next_time_step)
        replay_buffer.add_batch(traj)

        time_step = next_time_step
        collect_policy_state = action_step.state

    # Sample a batch of data from the replay buffer
    experience, _ = next(iterator)

    # Train the agent
    train_loss = agent.train(experience)

    # Print the loss at certain intervals
    if iteration % 200 == 0:  # Use the new variable name here
        print(f"Iteration: {iteration}, Loss: {train_loss.loss.numpy()}")




Iteration: 0, Loss: 1.0033327341079712
Iteration: 200, Loss: 1.0522117614746094


KeyboardInterrupt: ignored